#### **Cargando dimensiones**

**DIMENSION CLIENTE**

In [ ]:
import psycopg2
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Ejecutar la consulta para insertar datos en la tabla `dim_cliente`
cursor.execute("""
    INSERT INTO "CuboMultidimensional".dim_cliente (tipo_cliente)
    SELECT DISTINCT Tipo_de_Cliente
    FROM transaccional.Clientes;
""")

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()


**DIMENSION PARADA**

In [ ]:
#### **Cargamos dimension parada**
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Ejecutar la consulta para insertar datos en la tabla `dim_parada`
cursor.execute("""  
    INSERT INTO "CuboMultidimensional".dim_parada (parada, barrio, localidad, provincia)
    SELECT DISTINCT 
        p.Nombre AS parada,
        u.Barrio AS barrio,
        u.Localidad AS localidad,
        u.Provincia AS provincia
    FROM transaccional.Parada p
    JOIN transaccional.Ubicacion u ON p.ID_Ubicacion = u.ID_Ubicacion;
""")

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()


**DIMENSION COLECTIVO**

In [ ]:
#### **Cargamos dimension colectivo**
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Ejecutar la consulta para insertar datos en la tabla `dim_colectivo`
cursor.execute("""
    INSERT INTO "CuboMultidimensional".dim_colectivo (numero_colectivo, matricula)
    SELECT DISTINCT CAST(li.nombre AS VARCHAR), c.Matricula
    FROM transaccional.Colectivo c
    JOIN transaccional.linea_vehiculo l ON c.id_colectivo = l.vehiculo_id 
    JOIN transaccional.linea li ON l.linea_id = li.id_linea;
""")

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()


**DIMENSION FECHA**

In [ ]:
#### **Cargamos dimension fecha**
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Ejecutar la consulta para insertar datos en la tabla `dim_fecha`
cursor.execute("""
    INSERT INTO "CuboMultidimensional".dim_fecha (hora, dia, mes, ano)
    SELECT DISTINCT 
        EXTRACT(HOUR FROM Fecha_llegada)::integer AS hora,
        EXTRACT(DAY FROM Fecha_llegada) AS dia,
        EXTRACT(MONTH FROM Fecha_llegada) AS mes,
        EXTRACT(YEAR FROM Fecha_llegada) AS ano
    FROM transaccional.Clientes;
""")

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()

#### **Consulta intermedia del hecho**

In [23]:
import psycopg2
import pandas as pd

# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)

# Consulta SQL para obtener fecha de llegada y código postal
query = """
SELECT
    AVG(EXTRACT(EPOCH FROM (c.Fecha_salida - c.Fecha_llegada)) / 60) AS avg_tiempo_espera,
    COUNT(c.ID_Cliente) AS cantidad_personas_en_parada,
    NULL AS id_clima,
    dc.id_cliente,
    co.id_colectivo,
    dp.id_parada,
    dp.localidad,
    df.hora, df.dia, df.mes, df.ano
FROM transaccional.Clientes c
JOIN "CuboMultidimensional".dim_cliente dc ON dc.tipo_cliente = c.Tipo_de_Cliente
JOIN transaccional.Parada p ON p.ID_Parada = c.ID_Parada
JOIN "CuboMultidimensional".dim_parada dp ON dp.parada = p.nombre
JOIN transaccional.Colectivo co ON co.ID_Colectivo = c.colectivo_id
JOIN "CuboMultidimensional".dim_fecha df 
    ON df.dia = EXTRACT(DAY FROM c.Fecha_llegada)
    AND df.mes = EXTRACT(MONTH FROM c.Fecha_llegada)
    AND df.ano = EXTRACT(YEAR FROM c.Fecha_llegada)
    AND df.hora = EXTRACT(HOUR FROM c.Fecha_llegada)
JOIN transaccional.linea_vehiculo lv ON c.colectivo_id = lv.vehiculo_id
-- Aquí se usa una subconsulta correlacionada para obtener la fecha_de_asignacion más cercana
WHERE lv.fecha_de_asignacion = (
    SELECT fecha_de_asignacion
    FROM transaccional.linea_vehiculo lv_sub
    WHERE lv_sub.vehiculo_id = c.colectivo_id
    ORDER BY ABS(EXTRACT(EPOCH FROM (c.fecha_salida - lv_sub.fecha_de_asignacion)))
    LIMIT 1
)
GROUP BY co.id_colectivo, dp.id_parada, dc.id_cliente, df.hora, df.dia, df.mes, df.ano;
"""

# Ejecuta la consulta y almacena el resultado en un DataFrame
df_clientes = pd.read_sql(query, conexion)

# Cierra la conexión después de obtener los datos

conexion.close()

C:\Users\yeyer\AppData\Local\Temp\ipykernel_484\1080848955.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clientes = pd.read_sql(query, conexion)


#### **Conexion a la api para obtener los datos historicos del clima**

In [24]:
import urllib.request
import json
import pandas as pd

# Establece el rango de fechas en la URL para obtener datos históricos
fecha_inicio = "2023-11-01"
fecha_fin = "2023-11-30"
ubicacion = "parana"
api_key = "63T8LGUCCM4NM8UZH9EBVZKBN"

# URL para los datos históricos
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{ubicacion}/{fecha_inicio}/{fecha_fin}?unitGroup=metric&key={api_key}&contentType=json"
ResultBytes = urllib.request.urlopen(url)

# Parsear los resultados como JSON
jsonData = json.load(ResultBytes)



HTTPError: HTTP Error 429: 

#### **Cargamos los datos de la conexion y lo cargamos en un dataframe**

In [26]:
# Crear una lista para almacenar los datos de cada hora
data = []

# Recorre los datos JSON y extrae la información horaria
for day in jsonData['days']:
    fecha = day['datetime']
    for hour in day['hours']:
        # Extrae la información horaria y agrégala a la lista como un diccionario
        data.append({
            "ubicacion": ubicacion,
            "fecha": fecha,
            "hora": hour['datetime'],
            "temperatura": hour['temp'],
            "condiciones": hour['conditions'],
            "humedad": hour['humidity'],
            "viento": hour['windspeed']
        })

# Convertir la lista de datos en un DataFrame
df_clima_historico = pd.DataFrame(data)

# Mostrar el DataFrame
print(df_clima_historico)

    ubicacion       fecha      hora  temperatura       condiciones  humedad  \
0      parana  2023-10-01  00:00:00         11.6             Clear    53.23   
1      parana  2023-10-01  01:00:00         11.3             Clear    54.53   
2      parana  2023-10-01  02:00:00         10.6             Clear    55.80   
3      parana  2023-10-01  03:00:00          9.9             Clear    57.95   
4      parana  2023-10-01  04:00:00          9.4             Clear    60.88   
..        ...         ...       ...          ...               ...      ...   
715    parana  2023-10-30  19:00:00         17.0  Partially cloudy    70.49   
716    parana  2023-10-30  20:00:00         16.9  Partially cloudy    69.35   
717    parana  2023-10-30  21:00:00         16.4  Partially cloudy    71.69   
718    parana  2023-10-30  22:00:00         15.9  Partially cloudy    71.42   
719    parana  2023-10-30  23:00:00         15.7          Overcast    72.67   

     viento  
0       1.0  
1       1.2  
2       0

#### **Procesamiento de los datos**

**Convertimos las columnas a datatime**

In [27]:
# Crear una nueva columna 'fecha_hora' combinando las columnas 'ano', 'mes', 'dia' y 'hora'
df_clientes['fecha_hora'] = pd.to_datetime(
	df_clientes['ano'].astype(str) + '-' +
	df_clientes['mes'].astype(str).str.zfill(2) + '-' +
	df_clientes['dia'].astype(str).str.zfill(2) + ' ' +
	df_clientes['hora'].astype(str).str.zfill(2) + ':00:00'
)

# Mostrar el DataFrame con la nueva columna
print(df_clientes[['fecha_hora']])

# Convertir las columnas 'fecha' y 'hora' a un solo datetime
df_clima_historico['fecha_hora'] = pd.to_datetime(df_clima_historico['fecha'] + ' ' + df_clima_historico['hora'])

# Mostrar el DataFrame con la nueva columna
print(df_clima_historico[['fecha_hora']])

               fecha_hora
0     2023-11-03 00:00:00
1     2023-11-04 00:00:00
2     2023-11-07 00:00:00
3     2023-11-12 00:00:00
4     2023-11-20 00:00:00
...                   ...
69595 2023-11-24 22:00:00
69596 2023-11-27 22:00:00
69597 2023-11-01 23:00:00
69598 2023-11-08 23:00:00
69599 2023-11-28 23:00:00

[69600 rows x 1 columns]
             fecha_hora
0   2023-10-01 00:00:00
1   2023-10-01 01:00:00
2   2023-10-01 02:00:00
3   2023-10-01 03:00:00
4   2023-10-01 04:00:00
..                  ...
715 2023-10-30 19:00:00
716 2023-10-30 20:00:00
717 2023-10-30 21:00:00
718 2023-10-30 22:00:00
719 2023-10-30 23:00:00

[720 rows x 1 columns]


**Buscamos en el dataframe clima historico de la api y sabemos la condicion sabiendo la fecha y hora - la ubicacion esta intrinsica**

In [29]:
# Realizar un merge entre df_clientes y df_clima_historico basado en la columna 'fecha_hora'
df_clientess = df_clientes.merge(df_clima_historico[['fecha_hora', 'condiciones']], on='fecha_hora', how='left')

# Renombrar la columna 'condiciones' a 'tipo_de_clima'
df_clientess.rename(columns={'condiciones': 'tipo_de_clima'}, inplace=True)

# Mostrar el DataFrame resultante
print(df_clientess[['fecha_hora', 'tipo_de_clima']])

               fecha_hora     tipo_de_clima
0     2023-11-03 00:00:00  Partially cloudy
1     2023-11-04 00:00:00          Overcast
2     2023-11-07 00:00:00             Clear
3     2023-11-12 00:00:00             Clear
4     2023-11-20 00:00:00             Clear
...                   ...               ...
69595 2023-11-24 22:00:00  Partially cloudy
69596 2023-11-27 22:00:00          Overcast
69597 2023-11-01 23:00:00             Clear
69598 2023-11-08 23:00:00             Clear
69599 2023-11-28 23:00:00             Clear

[69600 rows x 2 columns]


**Codificamos a numeros la condicion climatica**

In [30]:
df_clientess['tipo_de_clima'].unique()

df_clientess['tipo_de_clima'] = df_clientess['tipo_de_clima'].apply(lambda x: 'Rain' if 'Rain' in x else x)

# Mostrar los valores únicos de la columna 'tipo_de_clima'
print(df_clientess['tipo_de_clima'].unique())

# Crear un diccionario para mapear las condiciones climáticas a números
clima_mapping = {
    'Overcast': 1,
    'Clear': 2,
    'Partially cloudy': 3,
    'Rain': 4
}

# Aplicar el mapeo a la columna 'tipo_de_clima'
df_clientess['tipo_de_clima'] = df_clientess['tipo_de_clima'].map(clima_mapping)

# Mostrar los valores únicos de la columna 'tipo_de_clima' después del mapeo
print(df_clientess['tipo_de_clima'].unique())

['Partially cloudy' 'Overcast' 'Clear' 'Rain']
[3 1 2 4]


**Traducimos las condiciones y creamos una nueva columna con la codificacion de los condiciones**

In [31]:
print(df_clima_historico['condiciones'].unique())

df_clima_historico['condiciones'] = df_clima_historico['condiciones'].apply(lambda x: 'Rain' if 'Rain' in x else x)

# Mostrar los valores únicos de la columna 'tipo_de_clima'
print(df_clima_historico['condiciones'].unique())

# Diccionario de mapeo para traducir las condiciones climáticas al español
traduccion_clima = {
    'Overcast': 'Nublado',
    'Clear': 'Despejado',
    'Partially cloudy': 'Parcialmente nublado',
    'Rain': 'Lluvia'
}

# Aplicar el mapeo a la columna 'condiciones'
df_clima_historico['condiciones'] = df_clima_historico['condiciones'].apply(lambda x: ', '.join([traduccion_clima.get(cond, cond) for cond in x.split(', ')]))

# Mostrar el DataFrame con las condiciones traducidas
print(df_clima_historico[['fecha_hora', 'condiciones']])

###########################################################################################################################################################################

# Crear un diccionario para mapear las condiciones climáticas a números
clima_mapping_es = {
    'Nublado': 1,
    'Despejado': 2,
    'Parcialmente nublado': 3,
    'Lluvia': 4
}

# Aplicar el mapeo a la columna 'condiciones'
df_clima_historico['condiciones_numericas'] = df_clima_historico['condiciones'].map(clima_mapping_es)

# Mostrar el DataFrame con la nueva columna
print(df_clima_historico[['fecha_hora', 'condiciones', 'condiciones_numericas']])

['Clear' 'Partially cloudy' 'Overcast' 'Rain, Overcast'
 'Rain, Partially cloudy']
['Clear' 'Partially cloudy' 'Overcast' 'Rain']
             fecha_hora           condiciones
0   2023-11-01 00:00:00             Despejado
1   2023-11-01 01:00:00             Despejado
2   2023-11-01 02:00:00             Despejado
3   2023-11-01 03:00:00             Despejado
4   2023-11-01 04:00:00             Despejado
..                  ...                   ...
715 2023-11-30 19:00:00  Parcialmente nublado
716 2023-11-30 20:00:00  Parcialmente nublado
717 2023-11-30 21:00:00  Parcialmente nublado
718 2023-11-30 22:00:00  Parcialmente nublado
719 2023-11-30 23:00:00               Nublado

[720 rows x 2 columns]
             fecha_hora           condiciones  condiciones_numericas
0   2023-11-01 00:00:00             Despejado                      2
1   2023-11-01 01:00:00             Despejado                      2
2   2023-11-01 02:00:00             Despejado                      2
3   2023-11-01 03:

**Reenombramos la columna**

In [32]:
# Renombrar columnas para que coincidan con la tabla en la base de datos
df_clima_historico = df_clima_historico.rename(columns={
    'condiciones': 'tipo_de_clima',
    'condiciones_numericas': 'id_clima'
})

#### **Cargamos la dim_clima**

In [63]:
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Inserta cada fila del DataFrame en la tabla `dim_clima`
for _, row in df_clima_historico.iterrows():
    cursor.execute("""
        INSERT INTO "CuboMultidimensional".dim_clima (id_clima, condicion_climatica)
        VALUES (%s, %s)
        ON CONFLICT (id_clima) DO NOTHING;
    """, (row['id_clima'], row['tipo_de_clima']))

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()

#### **Cargamos en la tabla de hechos**

In [33]:
# Configuración de la conexión a la base de datos
conexion = psycopg2.connect(
    dbname="Parada_1",
    user="postgres",
    password="1234",
    host="localhost"
)
cursor = conexion.cursor()

# Inserta cada fila del DataFrame en la tabla `fact_transporte`
for _, row in df_clientess.iterrows():
    cursor.execute("""
        INSERT INTO "CuboMultidimensional".fact_transporte (
            avg_tiempo_espera, 
            cantidad_personas_en_parada, 
            id_cliente,
            id_clima, 
            id_colectivo, 
            id_parada, 
            hora,
            dia, 
            mes,
            ano
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (
        row['avg_tiempo_espera'], 
        row['cantidad_personas_en_parada'], 
        row['id_cliente'],
        row['tipo_de_clima'], 
        row['id_colectivo'], 
        row['id_parada'], 
        row['hora'],
        row['dia'],
        row['mes'],
        row['ano']    
    ))

# Confirmar los cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()